In [5]:
# De ser necesario
!pip install bokeh


In [11]:
import pandas as pd
import os
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, Div, DataTable, TableColumn, CustomJS, HoverTool
from bokeh.layouts import row, column
from bokeh.plotting import figure
from bokeh.palettes import Category10

# Habilitar la salida de Bokeh en el notebook
output_notebook()

# Cargar los datos
results_dir = "/content/drive/My Drive/TFM/resultados"
results_df_all_models = pd.read_csv(os.path.join(results_dir, "resultados_todos_modelos.csv"))
all_predictions_df = pd.read_csv(os.path.join(results_dir, "todas_las_predicciones.csv"))

# Renombrar la columna 'R^2' a 'R2' para evitar errores de sintaxis
results_df_all_models = results_df_all_models.rename(columns={'R^2': 'R2'})

# Lista de modelos
model_list = results_df_all_models['Modelo'].tolist()

# Colores para los modelos
colors = Category10[len(model_list)]

# Añadir los colores a la fuente de datos para los gráficos
source_rmse_r2 = ColumnDataSource(data=dict(Modelo=results_df_all_models['Modelo'].tolist(),
                                            RMSE=results_df_all_models['RMSE'].tolist(),
                                            R2=results_df_all_models['R2'].tolist(),
                                            Color=colors))  # Añadir los colores

# Tabla de RMSE y R²
columns = [TableColumn(field="Modelo", title="Modelo"),
           TableColumn(field="RMSE", title="RMSE"),
           TableColumn(field="R2", title="R²")]
rmse_r2_table = DataTable(source=source_rmse_r2, columns=columns, width=600, height=280,
                          selectable="checkbox", index_position=None,
                          row_height=25)

# Gráfico de RMSE en barras horizontales
p_rmse = figure(y_range=model_list, height=350, title="RMSE de Modelos Seleccionados",
                toolbar_location=None, tools="")
p_rmse.hbar(y='Modelo', right='RMSE', height=0.9, source=source_rmse_r2, color='Color')

# Añadir herramienta de hover para mostrar información adicional
p_rmse.add_tools(HoverTool(tooltips=[("Modelo", "@Modelo"), ("RMSE", "@RMSE")]))

# Gráfico de R² en barras horizontales
p_r2 = figure(y_range=model_list, height=350, title="R² de Modelos Seleccionados",
              toolbar_location=None, tools="")
p_r2.hbar(y='Modelo', right='R2', height=0.9, source=source_rmse_r2, color='Color')

# Añadir herramienta de hover para mostrar información adicional
p_r2.add_tools(HoverTool(tooltips=[("Modelo", "@Modelo"), ("R²", "@R2")]))

# Gráfico 3: Comparación de Predicciones vs Valores Reales con colores por modelo
all_predictions_long = all_predictions_df.melt(id_vars=['Valores Reales'],
                                               var_name='Modelo', value_name='Predicciones')
all_predictions_long['Modelo'] = all_predictions_long['Modelo'].str.replace('_Pred', '')

# Almacenar los datos originales para no perderlos
original_preds_data = dict(Valores_Reales=all_predictions_long['Valores Reales'].tolist(),
                           Predicciones=all_predictions_long['Predicciones'].tolist(),
                           Modelo=all_predictions_long['Modelo'].tolist(),
                           Color=[colors[model_list.index(m)] for m in all_predictions_long['Modelo']])

source_preds = ColumnDataSource(data=original_preds_data)

p_preds = figure(width=600, height=400, title="Comparación de Predicciones vs Valores Reales")
p_preds.scatter('Valores_Reales', 'Predicciones', source=source_preds, color='Color', legend_field='Modelo')

# Añadir herramienta de hover al gráfico 3
p_preds.add_tools(HoverTool(tooltips=[("Modelo", "@Modelo"), ("Valores Reales", "@Valores_Reales"), ("Predicciones", "@Predicciones")]))

# Añadir la línea de referencia en el gráfico 3
p_preds.line(x='Valores_Reales', y='Valores_Reales', source=source_preds, color='red', line_dash="dashed", legend_label="Línea de Identidad")

# Gráfico 4: Predicciones vs Valores Reales para el Mejor Modelo
best_model_name = results_df_all_models.iloc[results_df_all_models['RMSE'].idxmin()]['Modelo']
best_preds = all_predictions_df[[best_model_name + '_Pred', 'Valores Reales']].rename(columns={best_model_name + '_Pred': 'Predicciones'})

source_best_preds = ColumnDataSource(data=dict(Valores_Reales=best_preds['Valores Reales'].tolist(),
                                               Predicciones=best_preds['Predicciones'].tolist()))

p_best_preds = figure(width=600, height=400, title=f"Predicciones vs Valores Reales ({best_model_name}) - Mejor Modelo")
p_best_preds.scatter('Valores_Reales', 'Predicciones', source=source_best_preds)

# Añadir herramienta de hover al gráfico 4
p_best_preds.add_tools(HoverTool(tooltips=[("Valores Reales", "@Valores_Reales"), ("Predicciones", "@Predicciones")]))

# Añadir la línea de referencia en el gráfico 4
p_best_preds.line(x='Valores_Reales', y='Valores_Reales', source=source_best_preds, color='red', line_dash="dashed", legend_label="Línea de Identidad")

# Función JS para la interactividad entre los gráficos y la tabla
callback = CustomJS(args=dict(source_rmse_r2=source_rmse_r2, source_preds=source_preds, original_data=original_preds_data), code="""
    var selected_indices = cb_obj.indices;
    var rmse_data = source_rmse_r2.data;
    var preds_data = original_data;  // Usar los datos originales
    var new_preds_data = {Valores_Reales: [], Predicciones: [], Modelo: [], Color: []};

    // Si no hay selección, restablecer los datos completos
    if (selected_indices.length === 0) {
        new_preds_data = preds_data;
    } else {
        // Filtrar los datos según la selección
        for (var i = 0; i < selected_indices.length; i++) {
            var modelo_seleccionado = rmse_data['Modelo'][selected_indices[i]];
            for (var j = 0; j < preds_data['Modelo'].length; j++) {
                if (preds_data['Modelo'][j] == modelo_seleccionado) {
                    new_preds_data['Valores_Reales'].push(preds_data['Valores_Reales'][j]);
                    new_preds_data['Predicciones'].push(preds_data['Predicciones'][j]);
                    new_preds_data['Modelo'].push(preds_data['Modelo'][j]);
                    new_preds_data['Color'].push(preds_data['Color'][j]);
                }
            }
        }
    }

    source_preds.data = new_preds_data;
    source_preds.change.emit();
""")

# Añadir el callback a la tabla para interactividad
rmse_r2_table.source.selected.js_on_change('indices', callback)

# Layout del dashboard
layout = column(
    Div(text="<h1>Dashboard de Resultados de Modelos</h1>"),
    row(p_rmse, p_r2),
    rmse_r2_table,  # Tabla centrada
    row(p_preds, p_best_preds)  # Gráfico 3 y Gráfico 4
)

# Mostrar el dashboard
show(layout)


In [14]:
from bokeh.resources import CDN
from bokeh.embed import file_html
from bokeh.plotting import output_file, save
import os

# Definir el directorio de resultados
results_dir = "/content/drive/MyDrive/TFM/resultados"

# Asegurarte de que el directorio exista
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

# Guardar como HTML en la carpeta de Google Drive
output_file(os.path.join(results_dir, "graficos_bokeh.html"))
save(layout)

'/content/drive/MyDrive/TFM/resultados/graficos_bokeh.html'